In [1]:
import numpy as np
import imageio
import mat73
import os
import cv2
import random
import time
import csv
import sys

In [2]:
##global Variable default state
global csv_file #defines csv_file as a global variable
global pathtoHDRgt
global csvPath
global pathtoSDR
global pathtoHDRgen
global result
csv_name = 'pictureData.csv'
csvPath = './csv/'
pathtoHDRgt = './groundTruthHDR/'
pathtoSDR = './LDR/'
pathtoHDRgen = './generatedHDR/'
result = './results/'
global HDRgtPic
global SDRpic
global HDRgenPic

In [3]:
#Read Picture and return it

def readThePicture(picturepath):
    #  open ImageObject
    try:
        print('Reading <==== '+picturepath)
        img = cv2.imread(picturepath, cv2.IMREAD_UNCHANGED)# | cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH)
    except:
        print('There was an error while reading the picture')
        img = 0
    return img

In [4]:
def writeToCV(file):
    #https://docs.python.org/3/library/csv.html
    with open(csv_name, 'w', newline='') as f:
        csv_file = csv.writer(f)
        csv_file.writerows([file])

In [5]:
def readCSV():
    #https://docs.python.org/3/library/csv.html
    with open(csv_name, newline='') as f:
        reader = csv.reader(f)
        for row in reader:
            return(row)

In [6]:
def savePic(picture,fileName,extention,outPath): #saves the given array as a pictures to the given output path
    outPath = outPath+fileName+'.'+extention
    try:

        #print(picture.shape)
        print('Writing picture to ====> '+outPath)
        cv2.imwrite(outPath,picture)
        
    except:
        print('Failed while saving picture: '+fileName+' to '+ outPath+' sorry :(')
        print('--------------------')

In [14]:
def together():
    try:
        global SDRpic,HDRgtPic,HDRgenPic
        SDRpic_new = SDRpic
        if(HDRgtPic.shape != SDRpic.shape != HDRgenPic.shape):
            factor = int(HDRgtPic.shape[1]/SDRpic.shape[1])
            print('Scaling up factor SDR: '+str(factor))
            scale = (int(SDRpic.shape[1])*factor, int(SDRpic.shape[0])*factor)
            #print(scale)
            SDRpic_new = cv2.resize(SDRpic, scale, interpolation = cv2.INTER_AREA)
            #print('BEW'+str(SDRpic_new.shape))
            SDRpic_new = np.clip(((SDRpic_new/((2**8)-1))*((2**16)-1)),0,((2**16)-1)).astype(np.uint16) ##scaling up to 16 bit png
            together = np.hstack((SDRpic_new,HDRgenPic,HDRgtPic))
        together = np.hstack((SDRpic_new,HDRgenPic,HDRgtPic))
        print(together.shape)
    except:
        print('Fail Pictures do not have the same size')
        print('Ground Truth: '+str(HDRgtPic.shape))
        print('Low Res pic: ' +str(SDRpic.shape))
        print('Low Res pic: ' +str(SDRpic_new.shape))
        print('Generated Picture: ' +str(HDRgenPic.shape))
    return together 

In [8]:
#scale = (SDRpic.shape[0]*2,SDRpic.shape[1]*2)
#cv2.resize(SDRpic, (scale))
#together()

In [9]:
def psnr(img_orig, img_out, peak):
    mse = np.mean(np.square(img_orig - img_out))
    psnr = 10 * np.log10(peak * peak / mse)

In [15]:
def inputData():
    global csv_file #defines csv_file as a global variable
    global pathtoHDRgt
    global csvPath
    global pathtoSDR
    global pathtoHDRgen
    global result
    
    print('This script stiches three pictures together in the Order: ')
    print('Ground Trouth, SDR inputpicture, Generated HDR Picture')
    result = input('Enter path for the results (default: '+result+'): ') or result
    
    if not os.path.exists(result):
        os.mkdir(result)
    csv_name = input('Enter the .csv file name (default: pictureData): ') or ('pictureData') +('.csv')
    csvPath = input('Enter the path to save Path file (deflaut: '+csvPath+'): ') or ('./csv/')
    if not os.path.exists(csvPath):
        os.mkdir(csvPath)
    pathtoHDRgt = input('Enter the path to the Ground Truth Picture (deflaut: '+pathtoHDRgt+') ') or pathtoHDRgt
    if not os.path.exists(pathtoHDRgt):
        os.mkdir(pathtoHDRgt)
    pathtoSDR = input('Enter the path to the SDR inputpicture (deflaut: '+pathtoSDR+') ') or pathtoSDR
    if not os.path.exists(pathtoSDR):
        os.mkdir(pathtoSDR)
    pathtoHDRgen = input('Enter the path to the Generated HDR Picture (deflaut: '+pathtoHDRgen+') ') or pathtoHDRgen
    if not os.path.exists(pathtoHDRgen):
        os.mkdir(pathtoHDRgen)
    
    assert os.path.exists(csvPath), "I did not find the path "+str(csvPath)
    csv_file = open(csv_name, 'w')
    csv_file = csv.writer(csv_file)

In [24]:
#finding the rigth picture pairs in paths
#global HDRgtPic

#global HDRgenPic
i = 0
inputData()
try:
    while( (sum(1 for f in os.listdir(pathtoHDRgen)))-1 >= i ):
        if((os.listdir(pathtoHDRgen)[i].split('.')[1]) != 'png'):
            i = i + 1    
        else:
            print(i)
            currentFileName = os.listdir(pathtoHDRgen)[i]
            i = i+1
            picName = currentFileName.split('.')[0]
            HDRgenPic = readThePicture(pathtoHDRgen+currentFileName)
            #search for the right picture with the same name but different ending
            foundFileSDR = False
            posSDR = 0
            while(foundFileSDR != True):
                if(picName == (os.listdir(pathtoSDR)[posSDR].split('.')[0])):
                    currentFileName = os.listdir(pathtoSDR)[posSDR]
                    SDRpic = readThePicture(str(pathtoSDR+currentFileName))*2
                    foundFileSDR = True
                posSDR = posSDR + 1

            foundFileGT = False
            posGT = 0
            while(foundFileGT != True):
                if(picName == (os.listdir(pathtoHDRgt)[posGT].split('.')[0])):
                    currentFileName = os.listdir(pathtoHDRgt)[posGT]
                    HDRgtPic = readThePicture(str(pathtoHDRgt+currentFileName))
                    #HDRgtPic = np.clip((HDRgtPic*((2**6)-1)).astype(np.uint16),0,((2**16)-1)) # brings the Datas of 10 bit in the visualt 16 bit png range
                    foundFileGT = True
                posGT = posGT + 1
            picTure = together()
            #print(picTure.shape+'h')
            savePic(picTure,(str(i)+'Result_'+picName),'png',result)
            #if(colorSpaceAnal == True ):
            #    savePic(picTure[:,:,0]*255,(str(i)+'Result_ColorSpace'+picName),'png',result)
             #   savePic(picTure[],(str(i)+'Result_ColorSpace'+picName),'png',result)
              #  savePic(picTure[],(str(i)+'Result_ColorSpace'+picName),'png',result)
        
print('Finished and it took: '+str((time.time() - start_time)/60)+'minutes')
except: 
    print('There was an error while finding the pictures to compare')
    print('Picture name to find: '+picName)

This script stiches three pictures together in the Order: 
Ground Trouth, SDR inputpicture, Generated HDR Picture
Enter path for the results (default: ./results/): 
Enter the .csv file name (default: pictureData): 
Enter the path to save Path file (deflaut: ./csv/): 
Enter the path to the Ground Truth Picture (deflaut: ./groundTruthHDR/) 
Enter the path to the SDR inputpicture (deflaut: ./LDR/) 
Enter the path to the Generated HDR Picture (deflaut: ./generatedHDR/) 
0
Reading <==== ./generatedHDR/000019.png
Reading <==== ./LDR/000019.png
Reading <==== ./groundTruthHDR/000019.png
Scaling up factor SDR: 2
(2160, 11520, 3)
Writing picture to ====> ./results/1Result_000019.png
1
Reading <==== ./generatedHDR/000018.png
Reading <==== ./LDR/000018.png
Reading <==== ./groundTruthHDR/000018.png
Scaling up factor SDR: 2
(2160, 11520, 3)
Writing picture to ====> ./results/2Result_000018.png
2
Reading <==== ./generatedHDR/000020.png
Reading <==== ./LDR/000020.png
Reading <==== ./groundTruthHDR/000

In [12]:
#savePic(together(),(str(i)+'Result_'+picName),'png',result)


In [13]:
HDRgtPic.shape[1]/SDRpic.shape[1]

2.0

In [22]:
(os.listdir(pathtoHDRgen)[i]).split('.')[1]

'png'